In [1]:
#- Morgen testen, ob der download von den pop daten mit dem flex namen auch funktioniert

In [2]:
import pandas as pd
import numpy as np
import os
import requests
import json


In [3]:
def data_import():
    """
    Import der Daten aus allen Dateien, die mit 'waqi-covid-' anfangen.
    Entfernen von Duplikaten und Umbenennung bestimmter Spaltenwerte.
    Zusammenführung der DataFrames.
    """
    data_folder = './data/'
    all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-') and f.endswith('.csv') or f == 'airquality-covid19-cities.json']
    dataframes = []

    if not all_files:
        print("Keine Dateien gefunden.")
        return None

    for file in all_files:
        file_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(file_path, comment='#')

            if "Specie" not in df.columns:
                print(f"Spalte 'Specie' fehlt in {file}")
                continue

            df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
            df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
            df = df.drop_duplicates()

            if df.empty:
                print(f"{file} enthält nach Duplikat-Entfernung keine Daten mehr.")
                continue

            dataframes.append(df)
        except Exception as e:
            print(f"Fehler beim Verarbeiten von {file}: {e}")

    if not dataframes:
        print("Keine gültigen Daten vorhanden.")
        return None

    return pd.concat(dataframes, ignore_index=True)

In [4]:
df=data_import()
print(df)

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.
                Date Country        City    Specie  count     min     max  \
0         2015-01-06      KR      Jeonju        co    124     0.1    12.3   
1         2015-01-22      KR      Jeonju        co    116     4.5    10.0   
2         2015-03-30      KR      Jeonju        co    118     1.2    11.2   
3         2015-05-27      KR      Jeonju        co     93     2.3     5.6   
4         2015-02-03      KR      Jeonju        co    133     4.5    13.4   
...              ...     ...         ...       ...    ...     ...     ...   
14257913  2024-10-05      MN  Ulan Bator  pressure     46  1027.0  1031.0   
14257914  2025-02-17      MN  Ulan Bator  pressure    264  1025.0  1027.0   
14257915  2025-02-26      MN  Ulan Bator  pressure    264  1002.0  1016.0   
14257916  2024-04-04      MN  Ulan Bator  pressure    240  1016.0  1019.0   
14257917  2024-05-15    

In [5]:
def geo_data(df):
    """
    Fügt die Geodaten zu den Städten hinzu
    """
    df = df.copy()

    # JSON-Datei laden mit Fehlerbehandlung
    try:
        with open('./data/airquality-covid19-cities.json', 'r', encoding='utf-8') as file:
            geodata = json.load(file)
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Fehler beim Laden der JSON-Datei: {e}")
        return df

    geodata = geodata.get("data", [])

    # Erstellen eines DataFrames mit Städten und Geokoordinaten
    df_places = pd.DataFrame([
        {
            "City": entry.get("Place", {}).get("name"),
            "Latitude": entry.get("Place", {}).get("geo", [None, None])[0],
            "Longitude": entry.get("Place", {}).get("geo", [None, None])[1]
        }
        for entry in geodata if "Place" in entry and "geo" in entry.get("Place", {})
    ])

    # Entferne Zeilen mit fehlenden Stadtnamen
    df_places.dropna(subset=["City"], inplace=True)

    # Standardisiere Stadtnamen
    df_places["City"] = df_places["City"].str.lower().str.strip()
    df["City"] = df["City"].str.lower().str.strip()

    # Zusammenführen der beiden DataFrames über "City"
    df = df.merge(df_places, on="City", how="left")

    # Überprüfung auf fehlende Geodaten
    fehlende_staedte = df[df["Latitude"].isna()]["City"].unique()
    if fehlende_staedte.size > 0:
        print(f"Keine Geodaten gefunden für: {', '.join(fehlende_staedte)}")

    return df

In [6]:
df = geo_data(df)
print(df.sample(10))

Keine Geodaten gefunden für: washington d.c.
                Date Country      City      Specie  count     min     max  \
11269325  2023-06-11      CN      wuxi        pm10    119    13.0    64.0   
5523550   2020-07-17      TR   antakya        pm10     24    28.0    35.0   
11500873  2023-08-30      US  honolulu    pressure     48  1016.5  1019.2   
7958225   2021-07-10      CN    haikou  wind-speed    147     0.5     6.6   
13668752  2024-08-26      FR  besançon          o3     10     5.9    36.5   
184450    2015-04-29      US    queens          o3     48    16.8    40.0   
8122962   2021-10-27      FR     nancy    humidity    168    59.0   100.0   
1743650   2018-05-20      BA     tuzla    pressure     82     0.0     0.0   
10957200  2023-06-11      RS       niš    humidity     48    44.5   100.0   
2296344   2019-05-06      US    austin    pressure     56  1009.5  1015.8   

          median  variance  Latitude  Longitude  
11269325    29.0   1868.08  31.56887  120.28857  
5523550

In [11]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df["City"] = df["City"].str.lower().str.strip()
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

    df = geo_data(df)

    df = weather_data(df)

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    return df



In [12]:
df.sample(10)

,Date,Country,City,Specie,count,min,max,median,variance,Latitude,Longitude
499693,2016-04-10,CN,nanjing,pm10,239,55.0,190.0,86.0,2607.26,32.06167,118.77778
210023,2015-01-08,JP,kumamoto,pm10,300,1.0,36.0,13.0,388.20,32.80589,130.69181
1963955,2019-03-26,TR,kayseri,pm10,68,4.0,76.0,31.0,2222.46,38.73222,35.48528
11549837,2023-07-10,AU,newcastle,temperature,142,8.5,19.9,12.7,94.67,-32.92953,151.78010
2466876,2019-06-22,CN,hangzhou,pressure,312,1003.0,1007.0,1006.0,8.54,30.29365,120.16142
10050520,2022-11-07,CN,wuxi,co,84,3.7,18.1,8.2,142.89,31.56887,120.28857
1943538,2019-01-26,FR,paris,wind-gust,384,0.8,20.4,9.8,145.67,48.85341,2.34880
8594842,2021-11-03,NL,utrecht,pressure,120,986.4,1005.0,989.2,545.17,52.09083,5.12222
9953723,2022-11-23,ZA,vereeniging,o3,102,0.5,24.2,12.3,460.95,-26.67313,27.92615
11911752,2023-10-18,CA,winnipeg,no2,13,0.2,2.7,0.6,5.43,49.88440,-97.14704


In [8]:
from meteostat import Daily, Stations
from datetime import datetime
import pandas as pd

def weather_data(df):
    """
    Ruft Wetterdaten für Städte im DataFrame ab und integriert sie.
    """
    df = df.copy()

    # Städte extrahieren und Duplikate entfernen
    cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

    # Zeitspanne festlegen
    start = datetime(2015, 1, 1)
    end = datetime(2024, 12, 31)

    # DataFrame für alle Städte vorbereiten
    all_data = pd.DataFrame()

    # Wetterdaten für jede Stadt abrufen und hinzufügen
    for _, city in cities.iterrows():
        try:
            # Nächste Wetterstation suchen
            stations = Stations().nearby(city['Latitude'], city['Longitude'])
            station = stations.fetch(1)

            if not station.empty:
                station_id = station.index[0]

                # Tägliche Wetterdaten abrufen
                data = Daily(station_id, start, end).fetch()

                # NaN-Daten rausfiltern
                data.dropna(how='all', inplace=True)

                if not data.empty:
                    # Stadtname hinzufügen
                    data["City"] = city["City"]

                    # Daten in den Gesamtdaten-Frame einfügen
                    all_data = pd.concat([all_data, data])

        except Exception as e:
            print(f"⚠️ Fehler beim Abrufen der Daten für {city['City']}: {e}")

    # Index zurücksetzen
    all_data.reset_index(inplace=True)

    # Spalte 'time' umbenennen in 'Date'
    all_data.rename(columns={'time': 'Date'}, inplace=True)

    # Standardisiere den Stadtnamen
    all_data["City"] = all_data["City"].str.lower().str.strip()

    print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

    # Konvertiere die 'Date'-Spalte in beiden DataFrames zu String-Format
    # df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

    # Berechne den Anteil der NaN-Werte pro Spalte
    missing_percentage = all_data.isna().mean() * 100
    # Lösche Spalten mit mehr als 80% NaN-Werten
    all_data = all_data.loc[:, missing_percentage <= 80]

    # Zusammenführen der beiden DataFrames über "City" und "Date"
    df = pd.merge(df, all_data, on=["City", 'Date'], how="left")

    return df


In [14]:
df = data_cleaning(df)


Keine Geodaten gefunden für: washington d.c.
✅ Wetterdaten gesammelt für 578 Städte
✅ Datei wurde gespeichert: ./data/cleaned_data.csv


In [15]:
print(df.sample(10).T)

                  331432      235178      1403671     1306809     530657   \
Date           2018-05-21  2017-05-09  2024-05-20  2023-10-24  2019-11-06   
Country                MY          JP          PL          NP          AT   
City           alor setar     okayama      zabrze     pokhara        linz   
aqi                  20.0         NaN         NaN         NaN         NaN   
co                    NaN         4.5         NaN         NaN         0.1   
d                     NaN         NaN         NaN         NaN         NaN   
dew                   NaN         NaN         NaN        14.1         5.0   
humidity             88.0         NaN        82.6        78.1        87.0   
mepaqi                NaN         NaN         NaN         NaN         NaN   
neph                  NaN         NaN         NaN         NaN         NaN   
no2                   NaN        20.4         NaN         NaN        15.1   
o3                    NaN        26.4         NaN         NaN         NaN   

In [10]:
#WEtterdaten integrieren


from meteostat import Daily, Stations
from datetime import datetime

df = df_copy()

# Städte
cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

# Zeitspanne festlegen
start = datetime(2015, 1, 1)
end = datetime(2024, 12, 31)

# DataFrame für alle Städte vorbereiten
all_data = pd.DataFrame()

# Daten für jede Stadt abrufen und hinzufügen
for _, city in cities.iterrows():
    # Nächste Wetterstation suchen
    stations = Stations().nearby(city['Latitude'], city['Longitude'])
    station = stations.fetch(1)

    if not station.empty:
        station_id = station.index[0]

        # Tägliche Wetterdaten abrufen
        data = Daily(station_id, start, end).fetch()

        # Nan-Daten rausfiltern
        data.dropna(how='all', inplace=True)

        if not data.empty:
            # Stadtname hinzufügen
            data["City"] = city["City"]

            # Daten in den Gesamtdaten-Frame einfügen
            all_data = pd.concat([all_data, data])


# Index zurücksetzen
all_data.reset_index(inplace=True)

# Spalte time umbennen in Date
all_data.rename(columns={'time': 'Date'}, inplace=True) 

# Standardisiere den Stadtnamen für eine bessere Übereinstimmung
all_data["City"] = all_data["City"].str.lower().str.strip()

print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

# # Ergebnis anzeigen
# print(all_data.head())

# Konvertiere die 'Date' Spalte in beiden DataFrames zu datetime
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

# Berechne den Anteil der NaN-Werte pro Spalte
missing_percentage = all_data.isna().mean() * 100

# Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
all_data = all_data.loc[:, missing_percentage <= 80]


# Zusammenführen der beiden DataFrames über "City"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")

NameError: name 'df_copy' is not defined

In [ ]:

print(df.shape)
df.sample(30)